In [2]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file

In [3]:
generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json")
test_instances = InstanceList()
for i in range(10):
    test_instances.append(instances[i * 50])

id_to_generator_name = {instance.id(): instance.filepath.parts[-2] for instance in test_instances}

In [4]:
PREFIX = "run-plain-90"
frames = []

for idx, db_path in enumerate(DATABASE_DIR.glob(f"{PREFIX}-*.db")):
    db = DB(db_path)
    results = pd.read_sql_query("SELECT * FROM results", db._conn)
    series = (
        results.loc[results["prefix"].str.startswith("test")]
        .groupby(["instance_id", "prefix"])["cost"]
        .min()
        .reset_index()
        .sort_values(["instance_id", "cost"])
        .groupby("instance_id")["cost"]
        .quantile(0.5, interpolation="lower")
        .rename(idx)
    )
    frames.append(series)

df = (
    pd.concat(frames, axis=1)
    .mean(axis=1)
    .round(2)
    .rename(PREFIX)
    .to_frame()
)
df.index = df.index.map(id_to_generator_name)
df.loc[generators].T

instance_id,cluster_netgen,compression,expansion,explosion,grid,cluster,implosion,linearprojection,rotation,uniform_portgen
run-plain-90,1.17,4.02,13.58,2.97,43.36,4.59,1.42,1.21,1.16,1.87


In [5]:
df = (
    pd.concat(frames, axis=1)
    .eq(100)
    .sum(axis=1)
    .rename(PREFIX)
    .to_frame()
)
df.index = df.index.map(id_to_generator_name)
df.loc[generators].T

instance_id,cluster_netgen,compression,expansion,explosion,grid,cluster,implosion,linearprojection,rotation,uniform_portgen
run-plain-90,0,0,1,0,4,0,0,0,0,0


In [10]:
results.loc[results["prefix"].str.startswith("config") & results["cached"].eq(0) & results["surrogate"].eq(0), "time"].sum()

123763.65757928848